In [ ]:
%matplotlib inline

from torch import nn
import torch.optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [ ]:
from livelossplot.input_plugins.pytorch_ignite import PlotLossesCallback 
from livelossplot.output_plugins.extrema_print import ExtremaPrint
from livelossplot.output_plugins.matplotlib import Matplotlib

In [ ]:
TRAIN_BATCH_SIZE = 100
TEST_BATCH_SIZE = 100

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(
        MNIST(download=True, root=".", transform=data_transform, train=True), batch_size=train_batch_size, shuffle=True
    )

    val_loader = DataLoader(
        MNIST(download=False, root=".", transform=data_transform, train=False), batch_size=val_batch_size, shuffle=False
    )
    return train_loader, val_loader

In [ ]:
model = Net()
train_loader, val_loader = get_data_loaders(TRAIN_BATCH_SIZE, TEST_BATCH_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
loss = torch.nn.NLLLoss()

In [ ]:
trainer = create_supervised_trainer(model, optimizer, loss)
metrics = metrics={
    'val_acc': Accuracy(),
    'val_loss': Loss(loss)
}
p_bar = ProgressBar()
p_bar.attach(trainer)
evaluator = create_supervised_evaluator(model, metrics=metrics)

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def validate(trainer):
    evaluator.run(val_loader)

In [ ]:
group_patterns = {'acccuracy': r'.*acc$', 'log-loss': r'.*loss$'}
callback = PlotLossesCallback(outputs=[Matplotlib(), ExtremaPrint()], group_patterns=group_patterns)
callback.attach(evaluator)

In [ ]:
trainer.run(train_loader, max_epochs=100)